### Embedding states |1>, |2>, |3>, |4>, & |5>

In [1]:
import numpy as np
from utils import DFS
modifiedStates = DFS().getModifiedAllStates()
states = DFS().getAllStates()

In [2]:
print(len(states))

8


In [3]:

print('ensuring states are normalized (aka <s|s> ~ 1)')
for s in states:
    print(np.dot(s,s))

print('-----------------------')
total = 0
for i in range(len(states)-1):
    for j in range(len(states)-1):
        if i != j:
            temp = np.dot(states[i],states[j])
            # print(f'<state{i+1}|state{j+1}> = {temp}')
            total += temp

orthoStatement = 'All 8 states are orthogonal with each other' if total <= 1e-14 else 'All 8 states are - NOT - orthogonal with each other'
print(orthoStatement)

ensuring states are normalized (aka <s|s> ~ 1)
1.0000000000000002
1.0
1.0
1.0
1.0000000000000004
1.0000000000000002
1.0000000000000002
1.0000000000000007
-----------------------
All 8 states are orthogonal with each other


### Above ^ states were shown to be orthogonal and form a computational basis to work with 
### --------------------------------------------------------
## Now below we are trying to use the fixed ansatz or circuit to see if we are encoding the circuit correctly

In [4]:
from scipy.linalg import expm
X = [[0,1],[1,0]]
Y = np.array([[0,-1j],[1j,0]], dtype=np.complex128)
Z = [[1,0],[0,-1]]
I = np.eye(2)
Udot = lambda s1, U, s2 : np.dot(np.conjugate(np.transpose(s1)),np.matmul(U,s2))

H_ex = (1/4)*(np.kron(X,X) + np.kron(Y,Y) + np.kron(Z,Z))

U_ex = lambda p : expm(-1j*np.pi*p*H_ex)

amp_sqrd = lambda c : np.real(c*np.conjugate(c))

inputStates, expectedStates = DFS().getInitialTargetStates()
modifiedInputStates, modifiedExpectedStates = DFS().getModifiedInitialTargetStates()
# evaluation function (minimization function)
def f_CNOT(U, modified=False):
    total_c_1 = 0
    for i in range(4):
        if modified:
            total_c_1 += amp_sqrd(Udot(modifiedExpectedStates[i],U,modifiedInputStates[i]))
        else:
            total_c_1 += amp_sqrd(Udot(expectedStates[i],U,inputStates[i]))
    total_c_2 = 0
    for i in range(4,len(inputStates)):
        if modified:
            total_c_2 += amp_sqrd(Udot(modifiedExpectedStates[i],U,modifiedInputStates[i]))
        else:
            total_c_2 += amp_sqrd(Udot(expectedStates[i],U,inputStates[i]))
    return np.sqrt(2-(1/4)*abs(total_c_1) - (1/4)*abs(total_c_2))

#### We need to do it in pennylane to make it easy to train ....
import pennylane as qml
from pennylane import numpy as np

np.random.seed(42)

def square_loss(targets, predictions):
    loss = 0
    for t, p in zip(targets, predictions):
        loss += (t - p) ** 2
    loss = loss / len(targets)
    return 0.5*loss

dev = qml.device('default.qubit', wires=6)

p1 = np.arccos(-1/np.sqrt(3))/np.pi
p2 = np.arcsin(1/3)/np.pi


def fixedCNOTCircuit():
    # We know this CNOT works
    # Figure 1 from "Universal Quantum Computation and Leakage Reduction in the 3-Qubit Decoherence Free Subsystem"
    qml.QubitUnitary(U_ex(p1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(-1/2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[1,2])
    qml.QubitUnitary(U_ex(-1/2), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(1), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[1,2])
    qml.QubitUnitary(U_ex(1/2), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(1), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[1,2])
    qml.QubitUnitary(U_ex(-1/2), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(-1/2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(1-p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(-p1), wires=[3,4])



@qml.qnode(dev)
def quantum_model(state_):
    qml.AmplitudeEmbedding(state_, wires=range(6))
    fixedCNOTCircuit()
    return qml.state()

def target_function(U_prime, modified=False):
    # approximated function
    return f_CNOT(U_prime, modified=modified)


print('NOT MODIFIED')
print("=====================================")
for i, state in enumerate(states): ############################# TODO: I think this needs to be fixed because it's outputting the wrong shit <<<<<<<<<<<<<<<<<<<-------------------------
    newstate = quantum_model(state)
    dot_val = np.dot(np.conjugate(np.transpose(newstate)), expectedStates[i])
    dot_sqrd = dot_val*np.conjugate(dot_val)
    print(f'for state{i+1}, we get |<expected_state|U_cnot|state{i+1}>|^2 = {np.real(dot_sqrd)}\t\t\t(should be = 1)')

op = qml.prod(fixedCNOTCircuit)() # finds the product for of these matrices
solved_U_cnot_matrix = qml.matrix(op)
print(f'If fixed matrix going into f_cnot = 0, then perfect! However, we get = {target_function(np.kron(I,solved_U_cnot_matrix))}')


print("MODIFIED")
print("=====================================")
for i, state in enumerate(modifiedStates):
    newstate = quantum_model(state)
    dot_val = np.dot(np.conjugate(np.transpose(newstate)), modifiedExpectedStates[i])
    dot_sqrd = dot_val*np.conjugate(dot_val)
    print(f'for state{i+1}, we get |<expected_state|U_cnot|state{i+1}>|^2 = {np.real(dot_sqrd)}\t\t\t(should be = 1)')

op = qml.prod(fixedCNOTCircuit)() # finds the product for of these matrices
solved_U_cnot_matrix = qml.matrix(op)
print(f'If fixed matrix going into f_cnot = 0, then perfect! However, we get = {target_function(np.kron(I,solved_U_cnot_matrix), modified=True)}')

NOT MODIFIED
for state1, we get |<expected_state|U_cnot|state1>|^2 = 0.06249999999999972			(should be = 1)
for state2, we get |<expected_state|U_cnot|state2>|^2 = 0.06250000000000011			(should be = 1)
for state3, we get |<expected_state|U_cnot|state3>|^2 = 0.080950376732586			(should be = 1)
for state4, we get |<expected_state|U_cnot|state4>|^2 = 0.08095037673258584			(should be = 1)


for state5, we get |<expected_state|U_cnot|state5>|^2 = 0.0625			(should be = 1)
for state6, we get |<expected_state|U_cnot|state6>|^2 = 0.06250000000000022			(should be = 1)
for state7, we get |<expected_state|U_cnot|state7>|^2 = 0.09876543209876552			(should be = 1)
for state8, we get |<expected_state|U_cnot|state8>|^2 = 0.09876543209876545			(should be = 1)
If fixed matrix going into f_cnot = 0, then perfect! However, we get = 1.3086699928923402
MODIFIED
for state1, we get |<expected_state|U_cnot|state1>|^2 = 0.06249999999999989			(should be = 1)
for state2, we get |<expected_state|U_cnot|state2>|^2 = 0.06250000000000006			(should be = 1)
for state3, we get |<expected_state|U_cnot|state3>|^2 = 0.08095037673258602			(should be = 1)
for state4, we get |<expected_state|U_cnot|state4>|^2 = 0.08095037673258594			(should be = 1)
for state5, we get |<expected_state|U_cnot|state5>|^2 = 0.06249999999999989			(should be = 1)
for state6, we get |<expected_state|U_cnot|state6>|^2 = 0.0625000000

In [5]:
import random

def fill_array_with_random_values(size):
    random_values = [random.uniform(-1, 1) for _ in range(size)]
    return random_values
array_size = 18
param = fill_array_with_random_values(array_size)



def variableCNOTCircuit(param):
    # We know this CNOT works
    # Figure 1 from "Universal Quantum Computation and Leakage Reduction in the 3-Qubit Decoherence Free Subsystem"
    qml.QubitUnitary(U_ex(p1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[0]), wires=[2,3])
    qml.QubitUnitary(U_ex(p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[1]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[2]), wires=[2,3])
    qml.QubitUnitary(U_ex(param[3]), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[4]), wires=[1,2])
    qml.QubitUnitary(U_ex(param[5]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[6]), wires=[2,3])
    qml.QubitUnitary(U_ex(param[7]), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[8]), wires=[1,2])
    qml.QubitUnitary(U_ex(param[9]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[10]), wires=[2,3])
    qml.QubitUnitary(U_ex(param[11]), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[12]), wires=[1,2])
    qml.QubitUnitary(U_ex(param[13]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[14]), wires=[2,3])
    qml.QubitUnitary(U_ex(param[15]), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[16]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[17]), wires=[2,3])
    qml.QubitUnitary(U_ex(1-p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(-p1), wires=[3,4])



In [6]:
def func_target(param, modified=False):
    solved_U_cnot_matrix = qml.matrix(variableCNOTCircuit)(param)
    return target_function(np.kron(I,solved_U_cnot_matrix), modified=modified)

In [7]:
import random

def fill_array_with_random_values(size):
    random_values = [random.uniform(-1, 1) for _ in range(size)]
    return random_values
array_size = 18
param = fill_array_with_random_values(array_size)

In [8]:
from scipy import optimize
bounds = optimize.Bounds(-1.0,1.0)
p_vals = optimize.minimize(func_target, param, method='L-BFGS-B',tol = 1e-5, bounds = bounds)

p_vals_mod = optimize.minimize(func_target, param, method='L-BFGS-B',tol = 1e-5, bounds = bounds, args=(True,))


In [9]:
print('NOT MODIFIED')
print(p_vals.x)
print('MODIFIED')
print(p_vals_mod.x)

NOT MODIFIED
[-4.99213636e-01  6.60475675e-05 -5.00792616e-01 -7.59026038e-02
  2.00804208e-01  6.02980397e-01 -2.41232108e-04  9.77859352e-01
  5.15510890e-01  9.62917661e-01 -2.09028570e-04 -5.46487373e-01
 -7.16510481e-01  6.96594958e-02  7.51589085e-01  5.07627573e-02
 -2.58425606e-04  2.48275326e-01]
MODIFIED
[-5.95011294e-01 -6.59694263e-04 -4.04254341e-01 -1.01495207e-01
  2.03775021e-01  9.85437047e-01  4.88261498e-04  9.90204610e-01
  4.59298069e-01  9.94144927e-01 -2.18910689e-04 -9.74157883e-01
 -6.62814073e-01  3.72589969e-02  6.67180459e-01  9.75984092e-02
 -6.48693219e-04  3.32925724e-01]


In [10]:
optimised_params = p_vals.x
mod_optimised_params = p_vals_mod.x

@qml.qnode(dev)
def quantum_model(state_):
    qml.AmplitudeEmbedding(state_, wires=range(6))
    variableCNOTCircuit(optimised_params)
    return qml.state()

@qml.qnode(dev)
def quantum_model_mod(state_):
    qml.AmplitudeEmbedding(state_, wires=range(6))
    variableCNOTCircuit(mod_optimised_params)
    return qml.state()

def target_function(U_prime, modified=False):
    # approximated function
    return f_CNOT(U_prime, modified=modified)


print('NOT MODIFIED')
print("=====================================")
for i, state in enumerate(states): ############################# TODO: I think this needs to be fixed because it's outputting the wrong shit <<<<<<<<<<<<<<<<<<<-------------------------
    newstate = quantum_model(state)
    dot_val = np.dot(np.conjugate(np.transpose(newstate)), expectedStates[i])
    dot_sqrd = dot_val*np.conjugate(dot_val)
    print(f'for state{i+1}, we get |<expected_state|U_cnot|state{i+1}>|^2 = {np.real(dot_sqrd)}\t\t\t(should be = 1)')

op = qml.prod(variableCNOTCircuit)(optimised_params) # finds the product for of these matrices
solved_U_cnot_matrix = qml.matrix(op)
print(f'If fixed matrix going into f_cnot = 0, then perfect! However, we get = {target_function(np.kron(I,solved_U_cnot_matrix))}')


print("MODIFIED")
print("=====================================")
for i, state in enumerate(modifiedStates): ############################# TODO: I think this needs to be fixed because it's outputting the wrong shit <<<<<<<<<<<<<<<<<<<-------------------------
    newstate = quantum_model_mod(state)
    dot_val = np.dot(np.conjugate(np.transpose(newstate)), modifiedExpectedStates[i])
    dot_sqrd = dot_val*np.conjugate(dot_val)
    print(f'for state{i+1}, we get |<expected_state|U_cnot|state{i+1}>|^2 = {np.real(dot_sqrd)}\t\t\t(should be = 1)')

op = qml.prod(variableCNOTCircuit)(mod_optimised_params) # finds the product for of these matrices
solved_U_cnot_matrix = qml.matrix(op)
print(f'If fixed matrix going into f_cnot = 0, then perfect! However, we get = {target_function(np.kron(I,solved_U_cnot_matrix), modified=True)}')

NOT MODIFIED
for state1, we get |<expected_state|U_cnot|state1>|^2 = 0.999999766150778			(should be = 1)
for state2, we get |<expected_state|U_cnot|state2>|^2 = 0.9438923513866582			(should be = 1)
for state3, we get |<expected_state|U_cnot|state3>|^2 = 1.9496754518495303e-07			(should be = 1)
for state4, we get |<expected_state|U_cnot|state4>|^2 = 2.3970300710503692e-08			(should be = 1)
for state5, we get |<expected_state|U_cnot|state5>|^2 = 0.9999995018951073			(should be = 1)
for state6, we get |<expected_state|U_cnot|state6>|^2 = 0.44664128968963734			(should be = 1)
for state7, we get |<expected_state|U_cnot|state7>|^2 = 8.16473697141858e-08			(should be = 1)
for state8, we get |<expected_state|U_cnot|state8>|^2 = 2.995876023458497e-08			(should be = 1)
If fixed matrix going into f_cnot = 0, then perfect! However, we get = 1.0000001650249675
MODIFIED
for state1, we get |<expected_state|U_cnot|state1>|^2 = 0.9999969375967799			(should be = 1)
for state2, we get |<expected_state|U_

In [11]:
p_vals = [param]
cost = [func_target(param)] 
opt = qml.AdagradOptimizer()
max_iterations = 100 
conv_tol = 0.1 

for n in range(max_iterations):
    param, prev_cost = opt.step_and_cost(func_target, param) # need to fix the cost function not optimising for fnot
    cost.append(func_target(param))
    p_vals.append(param)

    conv = np.abs(cost[-1] - prev_cost)
    if n % 10 == 0:
        print(f"Step = {n},  Cost function = {cost[-1]:.8f} ")
    if conv >= conv_tol:
        break

print("\n" f"Final value of the cost function = {cost[-1]:.8f} ")

Step = 0,  Cost function = 1.30332220 


/Users/danielmolina/opt/anaconda3/envs/qrl/lib/python3.10/site-packages/pennylane/_grad.py:110: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Step = 10,  Cost function = 1.30332220 
Step = 20,  Cost function = 1.30332220 
Step = 30,  Cost function = 1.30332220 
Step = 40,  Cost function = 1.30332220 
Step = 50,  Cost function = 1.30332220 
Step = 60,  Cost function = 1.30332220 
Step = 70,  Cost function = 1.30332220 
Step = 80,  Cost function = 1.30332220 
Step = 90,  Cost function = 1.30332220 

Final value of the cost function = 1.30332220 
